In [1]:
from datetime import datetime, timedelta
import psycopg2
import pandas as pd
from datetime import datetime, timedelta
from bson import ObjectId
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from sshtunnel import SSHTunnelForwarder
from dbutils.pooled_db import PooledDB

ssh_host = 'api.trustyapp.io'
ssh_username = 'ubuntu'
ssh_pem_file = './ng-prod.pem'
database_host = '127.0.0.1'  # SSH 터널을 통해 localhost로 접근
database_name = 'trusty'
database_username = 'server'
database_password = 'ygFAhsyTMDzB21VdLTnoq7djxwjDNzuc9yPGukm73otXER8ihD'

server = SSHTunnelForwarder(
    (ssh_host, 22),  # SSH 서버 주소와 포트
    ssh_username=ssh_username,
    ssh_pkey=ssh_pem_file,  # Private key 파일 경로
    remote_bind_address=(database_host, 5432)  # 원격 MySQL 서버 주소와 포트
)

server.start()

conn = psycopg2.connect(
    host='127.0.0.1',  # localhost를 통해 SSH 터널로 접속
    user=database_username,
    password=database_password,
    database=database_name,
    port=server.local_bind_port  # sshtunnel이 자동으로 할당한 포트
)
cursor = conn.cursor()

In [10]:
conn.rollback()
import pandas as pd
from datetime import datetime

qry = """
        SELECT *
        FROM "user" u
        WHERE u."helperId" IS NOT NULL OR u."installHelperId" IS NOT NULL
        """
cursor.execute(qry)
result = cursor.fetchall()

# DataFrame 생성 및 CSV 저장
column_names = [desc[0] for desc in cursor.description]
df = pd.DataFrame(result, columns=column_names)
df.to_csv('./부실그룹 파악.csv', index=False, encoding='utf-8-sig')

# 결과 확인
print(df.head())

                         createdAt                        updatedAt   id  \
0 2024-11-04 13:31:44.078665+00:00 2024-11-05 05:25:47.292433+00:00    8   
1 2024-11-27 10:51:59.505916+00:00 2025-01-08 09:28:03.167683+00:00  162   
2 2024-11-27 17:41:41.433018+00:00 2024-11-29 06:33:21.242762+00:00  179   
3 2024-11-27 11:01:04.189796+00:00 2025-01-19 15:28:50.976007+00:00  163   
4 2024-12-16 08:13:16.126773+00:00 2024-12-16 08:57:56.942935+00:00  384   

                                   uuid       username  \
0  4ebe1170-c8c4-4d29-b680-5aa2eef544d8  Angelopeters1   
1  10cafee6-133c-4d80-844b-0f4e4eede965          Norex   
2  eb678136-4d15-4bc8-971d-9ec05b5b861b       ifeoluwa   
3  25f877c5-b6dc-465d-ad9a-0169f844b9bf          Anson   
4  7156be25-a058-4d03-82cf-1821e7f5d315         Hannah   

                                            password bannedAt bannedReason  \
0  $2b$10$yUbGbn0bOKUc1P/ZqQdDcuv515QJlbpB2Ylck8W...     None         None   
1  $2b$10$IMnEyU2nX4ajGk8O3VB6GeHRkoFw

In [13]:
# 소개 로그가 찍혀있는 그룹 파악

conn.rollback()
import pandas as pd
from datetime import datetime

qry = """
        SELECT *
        FROM guarantee g
        LEFT JOIN friendship f ON g."userId" = f."userId"
        LEFT JOIN "user" u ON g."userId" = u."id"
        WHERE u."helperId" IS NOT NULL OR u."installHelperId" IS NOT NULL
        """
cursor.execute(qry)
result = cursor.fetchall()

# DataFrame 생성 및 CSV 저장
column_names = [desc[0] for desc in cursor.description]
df = pd.DataFrame(result, columns=column_names)
df.to_csv('./trust 상황 파악.csv', index=False, encoding='utf-8-sig')

# 결과 확인
print(df.head())

                         createdAt                        updatedAt  id  \
0 2025-01-25 06:19:03.590756+00:00 2025-01-30 14:24:21.672568+00:00  90   
1 2025-01-25 06:19:03.590756+00:00 2025-01-30 14:24:21.672568+00:00  90   
2 2025-01-25 06:19:03.590756+00:00 2025-01-30 14:24:21.672568+00:00  90   
3 2025-01-25 06:19:03.590756+00:00 2025-01-30 14:24:21.672568+00:00  90   
4 2025-01-25 06:19:03.590756+00:00 2025-01-30 14:24:21.672568+00:00  90   

  requestedAt                  lastEvaluatedAt evaluationResult  userId  \
0         NaT 2025-01-30 14:24:21.681000+00:00            trust     642   
1         NaT 2025-01-30 14:24:21.681000+00:00            trust     642   
2         NaT 2025-01-30 14:24:21.681000+00:00            trust     642   
3         NaT 2025-01-30 14:24:21.681000+00:00            trust     642   
4         NaT 2025-01-30 14:24:21.681000+00:00            trust     642   

   guaranteeTargetId                        createdAt  \
0                486 2025-01-16 14:56:49.

2025-01-31 21:24:24,311| ERROR   | Socket exception: An existing connection was forcibly closed by the remote host (10054)
